https://docs.ollama.com/api/openai-compatibility

In [ ]:
# 1. Install the required extraction tool (zstd)
!apt-get update && apt-get install -y zstd

# 2. Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [2]:
generation_model_id = "gemma2:9b-instruct-q5_0"
embedding_model_id = "qwen3-embedding:0.6b"

#### **ollama serve in back ground**

!nohub & -> & run and skip , any command between

!pkill -f ollama -> !pkill stop service

In [3]:
# 1. Force kill ANY existing Ollama background processes
!pkill -f ollama

In [4]:
# Starts the server on port 8000 and allows external connections
!nohup bash -c "OLLAMA_HOST=0.0.0.0:8000 OLLAMA_ORIGIN=* ollama serve" &
!sleep 5 && tail /content/nohup.out

nohup: appending output to 'nohup.out'
time=2026-02-26T20:11:59.139Z level=INFO source=images.go:480 msg="total unused blobs removed: 0"
time=2026-02-26T20:11:59.140Z level=INFO source=routes.go:1718 msg="Listening on [::]:8000 (version 0.17.1)"
time=2026-02-26T20:11:59.145Z level=INFO source=runner.go:67 msg="discovering available GPUs..."
time=2026-02-26T20:11:59.145Z level=INFO source=server.go:431 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --port 41673"
time=2026-02-26T20:12:00.118Z level=INFO source=server.go:431 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --port 43399"
time=2026-02-26T20:12:00.349Z level=INFO source=runner.go:106 msg="experimental Vulkan support disabled.  To enable, set OLLAMA_VULKAN=1"
time=2026-02-26T20:12:00.349Z level=INFO source=server.go:431 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --port 39959"
time=2026-02-26T20:12:00.350Z level=INFO source=server.go:431 msg="starting

In [5]:
!OLLAMA_HOST=0.0.0.0:8000 ollama pull {generation_model_id}
!OLLAMA_HOST=0.0.0.0:8000 ollama pull {embedding_model_id}

#### 0.0.0.0 accept from any host even if it is not 11434

bash -c "..." -> run any long complex command

In [5]:
# !nohup bash -c "OLLAMA_HOST=0.0.0.0:8000 OLLAMA_ORIGIN=* ollama run {generation_model_id} ollama run {embedding_model_id}"  &
# !sleep 5 && tail /content/nohup.out

%%bash -> no need to use !

Test Generation (Local Colab)

In [9]:
%%bash
curl http://localhost:8000/api/chat -d '{
  "model": "gemma2:9b-instruct-q5_0",
  "stream": false,
  "messages": [
    { "role": "user", "content": "ما عاصمة فرنسا؟" }
  ]
}'

{"model":"gemma2:9b-instruct-q5_0","created_at":"2026-02-26T20:17:51.091992444Z","message":{"role":"assistant","content":"عاصمة فرنسا هي **باريس**. 🇫🇷  \n"},"done":true,"done_reason":"stop","total_duration":8217340787,"load_duration":265956445,"prompt_eval_count":14,"prompt_eval_duration":7443069744,"eval_count":14,"eval_duration":447723977}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   512  100   366  100   146     44     17  0:00:08  0:00:08 --:--:--    91


Test Embeddings (Local Colab)

In [7]:
%%bash
curl http://localhost:8000/api/embeddings -d '{
  "model": "qwen3-embedding:0.6b",
  "prompt": "ما عاصمة مصر؟"
}'

{"embedding":[-0.011929064057767391,0.007997422479093075,0.0008353941957466304,0.005405955947935581,0.0407911017537117,-0.026510100811719894,-0.026249345391988754,0.0422142893075943,0.008788153529167175,0.020345192402601242,0.045505743473768234,-0.04953565075993538,0.04123896732926369,-0.00031448539812117815,-0.03378845006227493,0.03727716952562332,-0.014318693429231644,0.028116412460803986,0.07468735426664352,-0.04624500498175621,0.013744994066655636,-0.0019379580626264215,0.05203140527009964,0.0041766781359910965,0.027165811508893967,0.0008233179687522352,-0.05691652372479439,-0.0006387089961208403,0.018182680010795593,-0.02283954806625843,0.12022212892770767,0.06675630062818527,-0.05128192901611328,0.006017850711941719,-0.03594181686639786,-0.0006493636756204069,0.026367178186774254,-0.04984442889690399,-0.023604288697242737,-0.01866508461534977,-0.011449765413999557,-0.014730618335306644,0.008432799018919468,0.03800573572516441,-0.026725295931100845,-0.033523548394441605,0.05136620

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21833    0 21756  100    77    569      2  0:00:38  0:00:38 --:--:--  5448


## **Ngrok**

get the key from ngrok dashboard and put it in colab secrets

In [ ]:
!pip install pyngrok -q

In [11]:
from google.colab import userdata
from pyngrok import ngrok, conf

# Get your secret token
ngrok_auth = userdata.get('ngrok-colab')
conf.get_default().auth_token = ngrok_auth

# The port Ollama is running on
port = "8000"

# Disconnect any existing tunnels to avoid errors
ngrok.disconnect(port)

# Open the new tunnel
public_url = ngrok.connect(port).public_url
print(f"✅ Your public Ngrok URL is: {public_url}")
print(f"✅ Copy this into your local .env file as: {public_url}/v1")

✅ Your public Ngrok URL is: https://brycen-obliterative-camie.ngrok-free.dev
✅ Copy this into your local .env file as: https://brycen-obliterative-camie.ngrok-free.dev/v1


if ngrok tell me there is another agent work go to the dashboard and stop it from "agents"

**test it**

Test Everything via the Public URL



In [ ]:
import requests

# We use the URL generated in the previous cell
chat_url = f"{public_url}/api/chat"

payload = {
  "model": generation_model_id,
  "stream": False,
  "messages": [
    { "role": "user", "content": "ما هي أكبر مدينة في مصر؟" }
  ]
}

print("Sending request over the internet to Colab...")
response = requests.post(chat_url, json=payload)

if response.status_code == 200:
    print("\nSuccess! Here is the response:")
    print(response.json()['message']['content'])
else:
    print(f"\nError: {response.status_code}")
    print(response.text)